# Testing parse_assistant_message Function

This notebook demonstrates the functionality of the `parse_assistant_message` function from the `coder.py` module.

In [1]:
from autocoder.agent.coder import Coder
import byzerllm

# Create a mock instance just to access the parse_assistant_message function
llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")
coder = Coder(llm)


2024-12-06 21:31:20.746 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-12-06 21:31:20,788	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-12-06 21:31:20,803	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


## Test Case 1: Basic Text Content

In [2]:
message = "Hello, this is a simple text message"
result = coder.parse_assistant_message(message)
print("Test Case 1 Result:")
print(result)

Test Case 1 Result:
[{'type': 'text', 'content': 'Hello, this is a simple text message', 'partial': True}]


## Test Case 2: Single Tool Use (execute_command)

In [3]:
message = """
I'll execute the ls command:
<execute_command>
<command>ls -l</command>
</execute_command>
"""
result = coder.parse_assistant_message(message)
print("Test Case 2 Result:")
print(result)

Test Case 2 Result:
[{'type': 'text', 'content': "I'll execute the ls command:", 'partial': False}, {'type': 'tool_use', 'name': 'execute_command', 'params': {'command': 'ls -l'}, 'partial': False}, {'type': 'text', 'content': '', 'partial': True}]


## Test Case 3: Multiple Tools with Text In Between

In [4]:
message = """
First, let's read a file:
<read_file>
<path>test.txt</path>
</read_file>

Now, let's write to another file:

<write_to_file>
<path>output.txt</path>
<content>
Hello World!
This is a test.
</content>
</write_to_file>
"""
result = coder.parse_assistant_message(message)
print("Test Case 3 Result:")
print(result)

Test Case 3 Result:
[{'type': 'text', 'content': "First, let's read a file:", 'partial': False}, {'type': 'tool_use', 'name': 'read_file', 'params': {'path': 'test.txt'}, 'partial': False}, {'type': 'text', 'content': "Now, let's write to another file:", 'partial': False}, {'type': 'tool_use', 'name': 'write_to_file', 'params': {'path': 'output.txt', 'content': 'Hello World!\nThis is a test.'}, 'partial': False}, {'type': 'text', 'content': '', 'partial': True}]


## Test Case 4: Tool With Multiple Parameters

In [5]:
message = """
<search_files>
<path>src</path>
<regex>def\s+\w+</regex>
<file_pattern>*.py</file_pattern>
</search_files>
"""
result = coder.parse_assistant_message(message)
print("Test Case 4 Result:")
print(result)

Test Case 4 Result:
[{'type': 'text', 'content': '', 'partial': False}, {'type': 'tool_use', 'name': 'search_files', 'params': {'path': 'src', 'regex': 'def\\s+\\w+', 'file_pattern': '*.py'}, 'partial': False}, {'type': 'text', 'content': '', 'partial': True}]


## Test Case 5: Attempt Completion Tool

In [6]:
message = """
<attempt_completion>
<result>
Task completed successfully! Here are the results:
1. Files were processed
2. Data was analyzed
</result>
<command>cat output.txt</command>
</attempt_completion>
"""
result = coder.parse_assistant_message(message)
print("Test Case 5 Result:")
print(result)

Test Case 5 Result:
[{'type': 'text', 'content': '', 'partial': False}, {'type': 'tool_use', 'name': 'attempt_completion', 'params': {'result': 'Task completed successfully! Here are the results:\n1. Files were processed\n2. Data was analyzed', 'command': 'cat output.txt'}, 'partial': False}, {'type': 'text', 'content': '', 'partial': True}]


## Test Case 6: Partial Tool Use (Simulating Streaming)

In [7]:
message = """
<execute_command>
<command>ls
"""
result = coder.parse_assistant_message(message)
print("Test Case 6 Result (Partial Tool Use):")
print(result)

Test Case 6 Result (Partial Tool Use):
[{'type': 'text', 'content': '', 'partial': False}, {'type': 'tool_use', 'name': 'execute_command', 'params': {'command': 'ls'}, 'partial': True}]
